In [2]:
import pandas as pd
import requests
import json

In [4]:
response = requests.get("https://api.census.gov/data/timeseries/healthins/sahie?get=NIC_PT,NUI_PT,AGE_DESC,AGECAT,IPR_DESC,IPRCAT&for=county:*&in=state:37&time=2019").json()
response_df = pd.DataFrame(response)
response_df.head(50)

,0,1,2,3,4,5,6,7,8
0,NIC_PT,NUI_PT,AGE_DESC,AGECAT,IPR_DESC,IPRCAT,time,state,county
1,114466,19822,Under 65 years,0,All Incomes,0,2019,37,001
2,37624,11050,Under 65 years,0,<= 200% of Poverty,1,2019,37,001
3,47248,13355,Under 65 years,0,<= 250% of Poverty,2,2019,37,001
4,24827,7531,Under 65 years,0,<= 138% of Poverty,3,2019,37,001
5,74273,17315,Under 65 years,0,<= 400% of Poverty,4,2019,37,001
6,49446,9784,Under 65 years,0,138% to 400% of Poverty,5,2019,37,001
7,80092,17813,18 to 64 years,1,All Incomes,0,2019,37,001
8,21138,9942,18 to 64 years,1,<= 200% of Poverty,1,2019,37,001
9,27603,12002,18 to 64 years,1,<= 250% of Poverty,2,2019,37,001
